In [1]:
!pip install pyyaml h5py

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.18.0


In [3]:
!python --version

Python 3.11.13


In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [6]:
#piszemy funkcję create_model() która pozwoli utworzyć model sieci neuronowej o warstwie o gęstości 512
#z dropoutem 0.2, strata: kategoryczna entropia krzyżowa, metryka: accuracy
#funkcja zwraca model

def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [7]:
model = create_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
#użycie callbacks w punktach kontrolnych - utworzenie katalogu trening_1, w nim pliki cp_ckpt.weights.h5
#zdefiniowanie metody Checkpoint tylko z zapisem wag
#zdefiniowanie i uruchomienie treningu sieci z użyciem callbacks
checkpoint_path = "training_1/cp_ckpt.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(
    train_images,
    train_labels,
    epochs=12,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback]
)

Epoch 1/12
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4642 - loss: 1.6501
Epoch 1: saving model to training_1/cp_ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4825 - loss: 1.6054 - val_accuracy: 0.7840 - val_loss: 0.7284
Epoch 2/12
28/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8865 - loss: 0.4437
Epoch 2: saving model to training_1/cp_ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8859 - loss: 0.4406 - val_accuracy: 0.8310 - val_loss: 0.5409
Epoch 3/12
26/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9196 - loss: 0.3055
Epoch 3: saving model to training_1/cp_ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9192 - loss: 0.3059 - val_accuracy: 0.8530 - val_loss: 0.4763
Epoch 4/12
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9507 - loss: 0.2033
Epoch 4: saving model to training_1/cp_ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9507 - loss: 0.2035 - val_accuracy: 0.8620

In [10]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0402 - loss: 2.3659
Untrained model, accuracy:  5.90%


In [11]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

21/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8973 - loss: 0.3220 

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8854 - loss: 0.3644
Untrained model, accuracy: 86.20%


In [13]:
checkpoint_path = 'training_2/cp-{epoch:04d}.weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size #zapis co 5 epokę
)

model = create_model()



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
os.makedirs(checkpoint_dir, exist_ok=True)
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    train_images,
    train_labels,
    epochs=60,
    batch_size=batch_size,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0
)


Epoch 5: saving model to training_2/cp-0005.weights.h5

Epoch 10: saving model to training_2/cp-0010.weights.h5

Epoch 15: saving model to training_2/cp-0015.weights.h5

Epoch 20: saving model to training_2/cp-0020.weights.h5

Epoch 25: saving model to training_2/cp-0025.weights.h5

Epoch 30: saving model to training_2/cp-0030.weights.h5

Epoch 35: saving model to training_2/cp-0035.weights.h5

Epoch 40: saving model to training_2/cp-0040.weights.h5

Epoch 45: saving model to training_2/cp-0045.weights.h5

Epoch 50: saving model to training_2/cp-0050.weights.h5

Epoch 55: saving model to training_2/cp-0055.weights.h5

Epoch 60: saving model to training_2/cp-0060.weights.h5


In [15]:
model = create_model()
loss,acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0610 - loss: 2.3863
Untrained model, accuracy:  7.30%


In [22]:
model.load_weights('training_2/cp-0060.weights.h5')
loss,acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8988 - loss: 0.4006


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Untrained model, accuracy: 87.80%


In [23]:
#zapis całego modelu
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.h5')

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4798 - loss: 1.6528
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8737 - loss: 0.4573
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9377 - loss: 0.2666
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9369 - loss: 0.2267
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9564 - loss: 0.1748


In [24]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,052 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [25]:
loss,acc = new_model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8812 - loss: 0.3969
Untrained model, accuracy: 85.60%
